In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("7817_1.csv")

In [3]:
df.shape

(1597, 27)

In [4]:
df.head()

,id,asins,brand,categories,colors,dateAdded,dateUpdated,dimension,ean,keys,...,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,sizes,upc,weight
0,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,5.0,https://www.amazon.com/Kindle-Paperwhite-High-...,I initially had trouble deciding between the p...,"Paperwhite voyage, no regrets!",NaN,NaN,Cristina M,NaN,NaN,205 grams
1,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,5.0,https://www.amazon.com/Kindle-Paperwhite-High-...,Allow me to preface this with a little history...,One Simply Could Not Ask For More,NaN,NaN,Ricky,NaN,NaN,205 grams
2,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,4.0,https://www.amazon.com/Kindle-Paperwhite-High-...,I am enjoying it so far. Great for reading. Ha...,Great for those that just want an e-reader,NaN,NaN,Tedd Gardiner,NaN,NaN,205 grams
3,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,5.0,https://www.amazon.com/Kindle-Paperwhite-High-...,I bought one of the first Paperwhites and have...,Love / Hate relationship,NaN,NaN,Dougal,NaN,NaN,205 grams
4,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,5.0,https://www.amazon.com/Kindle-Paperwhite-High-...,I have to say upfront - I don't like coroporat...,I LOVE IT,NaN,NaN,Miljan David Tanic,NaN,NaN,205 grams


In [5]:
import spacy
nlp=spacy.load("en_core_web_sm")

In [6]:
custom_stop_words=["want","gift","enjoy","roku"]
for word in custom_stop_words:
    nlp.vocab[word].is_stop=True

In [7]:
def preprocess(t):
    finaltext=""
    text=nlp(t)
    for token in text:
        if token.is_stop or token.is_punct:
            continue
        finaltext+=token.lemma_+" "
    return finaltext

In [8]:
df_text=df['reviews.text'].apply(preprocess)

In [9]:
df_text.head()

0    initially trouble decide paperwhite voyage rev...
1    allow preface little history casual reader own...
2    enjoy far great read original Fire 2012 Fire e...
3    buy Paperwhites pleased constant companion sup...
4    upfront like coroporate hermetically closed st...
Name: reviews.text, dtype: object

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
vectorizer = TfidfVectorizer( max_features=1000, max_df=0.85)

In [12]:
tfidf_matrix = vectorizer.fit_transform(df_text)

In [13]:
feature_names = vectorizer.get_feature_names_out()
feature_names[200:300]

array(['connect', 'connection', 'connectivity', 'consider',
       'consideration', 'consistent', 'console', 'content', 'continue',
       'contrast', 'control', 'controller', 'convenient', 'conveniently',
       'cool', 'cord', 'core', 'correct', 'correctly', 'cost', 'couch',
       'couple', 'course', 'cover', 'crash', 'crazy', 'create', 'crisp',
       'current', 'currently', 'customer', 'daily', 'dallas', 'dangle',
       'dark', 'date', 'daughter', 'day', 'deal', 'decent', 'decide',
       'definitely', 'delete', 'deliver', 'demand', 'depend',
       'description', 'design', 'despite', 'detail', 'device',
       'difference', 'different', 'digital', 'directly', 'disappoint',
       'disappointed', 'discover', 'dislodge', 'display', 'documents',
       'dot', 'dots', 'download', 'dozen', 'drive', 'drop', 'dual',
       'dumb', 'durable', 'dvd', 'dvr', 'ear', 'earbud', 'early', 'earn',
       'earpod', 'earpods', 'ease', 'easily', 'easy', 'echo', 'edge',
       'edit', 'edition', 'e

In [14]:
from sklearn.decomposition import LatentDirichletAllocation

num_topics = 10 

lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=100)
lda_results = lda_model.fit_transform(tfidf_matrix)

In [21]:
topics = lda_model.print_topics()

AttributeError: 'LatentDirichletAllocation' object has no attribute 'print_topics'

In [15]:
lda_model.components_.shape

(10, 1000)

In [16]:
topicstore={}
i=0

In [17]:
for probs in lda_model.components_:
    most_prominent_word_idx=probs.argmax()
    most_prominent_word=feature_names[most_prominent_word_idx]
    print(f"Topic name {most_prominent_word}")
    topicstore[i]=most_prominent_word
    i+=1
    listofwords=[feature_names[index] for index in probs.argsort()[-10:][::-1]]
    print(listofwords)
    print()

Topic name read
['read', 'question', 'ask', 'tablet', 'plug', 'charger', 'usb', 'micro', 'kindle', 'port']

Topic name roku
['roku', 'comcast', 'tv', 'fire', 'box', 'apple', 'smartphone', 'content', 'samsung', 'feature']

Topic name early
['early', 'reader', 'response', 'secondary', 'immediate', 'far', 'base', 'perform', 'heavy', 'touch']

Topic name prime
['prime', 'cover', 'case', 'echo', 'dot', 'great', 'amazon', 'buy', 'tap', 'look']

Topic name good
['good', 'family', 'great', 'br', 'fit', 'perfect', 'love', 'worth', 'reviewer', 'bedroom']

Topic name hdx
['hdx', 'kindle', 'fire', 'device', 'blue', 'download', 'team', 'protector', 'capacity', 'amazon']

Topic name headphone
['headphone', 'ear', 'sound', 'bud', 'like', 'apple', 'earbud', 'feel', 'people', 'fall']

Topic name kindle
['kindle', 'tablet', 'screen', 'fire', 'book', 'read', 'paperwhite', 'prime', 'tv', 'amazon']

Topic name echo
['echo', 'tap', 'amazon', 'remote', 'alexa', 'voice', 'work', 'command', 'device', 'fire']



In [18]:
topicstore

{0: 'read',
 1: 'roku',
 2: 'early',
 3: 'prime',
 4: 'good',
 5: 'hdx',
 6: 'headphone',
 7: 'kindle',
 8: 'echo',
 9: 'speaker'}

In [19]:
dominant_topics=[topic_weights.argmax() for topic_weights in lda_results]

In [20]:
for i, dominant_topic in enumerate(dominant_topics):
    print(f"Review {i+1} belongs to the topic {topicstore[dominant_topic]}")

Review 1 belongs to the topic kindle
Review 2 belongs to the topic kindle
Review 3 belongs to the topic kindle
Review 4 belongs to the topic kindle
Review 5 belongs to the topic kindle
Review 6 belongs to the topic hdx
Review 7 belongs to the topic kindle
Review 8 belongs to the topic kindle
Review 9 belongs to the topic kindle
Review 10 belongs to the topic kindle
Review 11 belongs to the topic kindle
Review 12 belongs to the topic kindle
Review 13 belongs to the topic kindle
Review 14 belongs to the topic kindle
Review 15 belongs to the topic kindle
Review 16 belongs to the topic kindle
Review 17 belongs to the topic read
Review 18 belongs to the topic headphone
Review 19 belongs to the topic roku
Review 20 belongs to the topic kindle
Review 21 belongs to the topic kindle
Review 22 belongs to the topic kindle
Review 23 belongs to the topic kindle
Review 24 belongs to the topic kindle
Review 25 belongs to the topic kindle
Review 26 belongs to the topic kindle
Review 27 belongs to the 